In [4]:
import os
import sys
import numpy as np
import skimage
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
import mrcnn.model as modellib
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [5]:
# minimum input size = 128
class ShapesConfig(Config):
    # Give the configuration a recognizable name
    NAME = "skin_michael"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 16
    NUM_CLASSES = 1 + 2  # background + 2 types
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128
    RPN_ANCHOR_SCALES = (16,32,64,128,256)  # anchor side in pixels
    TRAIN_ROIS_PER_IMAGE = 8
    STEPS_PER_EPOCH = 626 // IMAGES_PER_GPU
    VALIDATION_STEPS = 626 // IMAGES_PER_GPU
    LEARNING_RATE = 0.001
    USE_MINI_MASK = False
config = ShapesConfig()

class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MAX_DIM = 128
inference_config = InferenceConfig()

In [6]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference",
                          config=inference_config,
                          model_dir=MODEL_DIR)

model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

keras version 2.2.5
tf version 1.12.0
Loading weights from  C:\Users\kuki\PycharmProjects\Mask_RCNN\logs\skin20201027T0132\mask_rcnn_skin_0015.h5
Re-starting from epoch 15


In [7]:
from PIL import Image
from skimage import io

## put folder path here to apply your model to classify
src = r'\\motherserverdw\Kyu_Sync\server for undergrads\Charlotte+Michael\fibroblast_detection_v1\complete\test\5619_Wirtz.Denis_OTS-19_5619-021_tissue_region_2_tile\image'
##
dst = os.path.join(src,'classified')
if not os.path.exists(dst): os.mkdir(dst)

images = [os.path.join(src,_) for _ in os.listdir(src) if _.endswith('png')]
idd = []
for original_image in images:
    original_image2 = skimage.io.imread(original_image)
    results = model.detect([original_image2], verbose=1)
    r = results[0]
    masks = r['masks']
    masks = np.moveaxis(masks,2,0)
    if len(masks)<1:
        continue
    maskzero=np.zeros(masks[0].shape)
    for mask,id in zip(masks,r['class_ids']):
        idd.append(id)
        maskzero[mask]=id
    im = Image.fromarray(maskzero)
    im.save(os.path.join(dst, os.path.basename(original_image).replace('png','tif')))

Processing 1 images
image                    shape: (100, 100, 3)         min:   16.00000  max:  255.00000  uint8
molded_images            shape: (1, 128, 128, 3)      min:  -99.80000  max:  150.10000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  128.00000  float64
anchors                  shape: (1, 4092, 4)          min:   -1.42535  max:    1.92141  float32
Processing 1 images
image                    shape: (100, 100, 3)         min:   34.00000  max:  255.00000  uint8
molded_images            shape: (1, 128, 128, 3)      min:  -80.80000  max:  150.10000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  128.00000  float64
anchors                  shape: (1, 4092, 4)          min:   -1.42535  max:    1.92141  float32
Processing 1 images
image                    shape: (100, 100, 3)         min:   51.00000  max:  255.00000  uint8
molded_images            shape: (1, 128, 128, 3)      min:  -62.80000  max:  150.1

In [8]:
# calculate accuracy of the model based on predicted label
srcc = r'\\kukissd\Kyu_Sync\server for undergrads\Stephen\train'
srcfolders = [_ for _ in os.listdir(srcc) if os.path.isdir(_)]
for srcfolder in srcfolders:
    src = os.path.join(srcfolder,'image')
    dst = os.path.join(src,'classified')
    if not os.path.exists(dst): os.mkdir(dst)

    images = [os.path.join(src,_) for _ in os.listdir(src) if _.endswith('png')]
    idd = []
    for original_image in images:
        original_image2 = skimage.io.imread(original_image)
        results = model.detect([original_image2], verbose=1)
        r = results[0]
        masks = r['masks']
        masks = np.moveaxis(masks,2,0)
        if len(masks)<1:
            continue
        maskzero=np.zeros(masks[0].shape)
        for mask,id in zip(masks,r['class_ids']):
            idd.append(id)
            maskzero[mask]=id
        im = Image.fromarray(maskzero)
        if id == 1:
            dst1 = os.path.join(dst,'fibroblast')
            if not os.path.exists(dst1): os.mkdir(dst1)
        else:
            dst1 = os.path.join(dst,'false_positive')
            if not os.path.exists(dst1): os.mkdir(dst1)
        im.save(os.path.join(dst1, os.path.basename(original_image).replace('png','tif')))
    print(idd)


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]